In [3]:
import pandas as pd
from sklearn.utils import shuffle
import numpy as np
import xgboost as xgb

xgb.set_config(verbosity=0)
import joblib
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [4]:
model_p = xgb.XGBRegressor(verbosity=0)
model_p.load_model("../../emotion_classifier/model_training/models/bandai_l2p_model.json")

model_a = xgb.XGBRegressor(verbosity=0)
model_a.load_model("../../emotion_classifier/model_training/models/bandai_l2a_model.json")

model_d = xgb.XGBRegressor(verbosity=0)
model_d.load_model("../../emotion_classifier/model_training/models/bandai_l2d_model.json")

scaler = joblib.load('../../emotion_classifier/model_training/datasets/scalers/standardizers/S_BANDAI_5frame.pkl') 

In [5]:
dataset = pd.read_csv('datasets/BANDAI_5frame.csv')
dataset.head()

,max_hand_distance,avg_l_hand_hip_distance,avg_r_hand_hip_distance,max_stride_length,avg_l_hand_chest_distance,avg_r_hand_chest_distance,avg_l_elbow_hip_distance,avg_r_elbow_hip_distance,avg_chest_pelvis_distance,avg_neck_chest_distance,...,r_foot_speed,neck_speed,l_hand_acceleration_magnitude,r_hand_acceleration_magnitude,l_foot_acceleration_magnitude,r_foot_acceleration_magnitude,neck_acceleration_magnitude,EMOTION_P,EMOTION_A,EMOTION_D
0,0.433596,0.130036,0.335475,0.538619,0.184473,0.310089,0.210679,0.093915,0.236151,0.223894,...,-0.137483,-0.139467,0.092920,0.196129,0.221155,0.137483,0.139467,0.05,-0.4,0.0
1,0.348051,0.145184,0.256784,0.455501,0.197954,0.284215,0.172405,0.104876,0.236151,0.223894,...,-0.072441,-0.095084,0.048935,0.032804,0.070625,0.072096,0.045111,0.05,-0.4,0.0
2,0.320294,0.206306,0.180224,0.380190,0.234406,0.259905,0.133006,0.127385,0.236151,0.223894,...,-0.039016,-0.093423,0.030819,0.043059,0.028182,0.036532,0.014232,0.05,-0.4,0.0
3,0.400389,0.287300,0.136974,0.319861,0.279208,0.239450,0.112632,0.156856,0.236151,0.223894,...,-0.036825,-0.101489,0.044534,0.054382,0.015679,0.012813,0.021506,0.05,-0.4,0.0
4,0.465923,0.356129,0.136171,0.309995,0.313229,0.224432,0.112950,0.186680,0.236151,0.223894,...,-0.054338,-0.117573,0.044670,0.036618,0.020074,0.022012,0.024961,0.05,-0.4,0.0


In [7]:
train_dataset = dataset.sample(frac=0.8, random_state=42)
test_dataset = dataset.drop(train_dataset.index)

print("No Training Samples:",train_dataset.shape[0])
print("No Test Samples:",test_dataset.shape[0])

train_dataset = shuffle(train_dataset)
test_dataset = shuffle(test_dataset)

No Training Samples: 62841
No Test Samples: 15710


In [8]:
train_emotions = pd.concat([train_dataset.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)
train_emotions_OG = train_emotions.copy()

test_emotions = pd.concat([test_dataset.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)
test_emotions_OG = test_emotions.copy()

In [9]:
train_dataset_scaled = scaler.transform(train_dataset)
test_dataset_scaled = scaler.transform(test_dataset)

In [10]:
train_emotions_p = model_p.predict(train_dataset_scaled)
train_emotions_a = model_a.predict(train_dataset_scaled)
train_emotions_d = model_d.predict(train_dataset_scaled)

for i in range(len(train_dataset_scaled)):
    train_emotions.iloc[i] = [train_emotions_p[i], train_emotions_a[i], train_emotions_d[i]]

train_emotions.head()

,EMOTION_P,EMOTION_A,EMOTION_D
16143,-0.095588,-0.549284,-0.145846
73434,0.800709,0.511273,0.151796
1274,0.596452,0.406101,0.104351
10930,-0.102860,-0.544729,-0.146453
77906,0.200181,-0.728823,-0.287045


In [11]:
train_emotions_OG.head()

,EMOTION_P,EMOTION_A,EMOTION_D
16143,-0.1,-0.55,-0.15
73434,0.8,0.50,0.15
1274,0.6,0.40,0.10
10930,-0.1,-0.55,-0.15
77906,0.2,-0.75,-0.30


In [12]:
test_emotions_p = model_p.predict(test_dataset_scaled)
test_emotions_a = model_a.predict(test_dataset_scaled)
test_emotions_d = model_d.predict(test_dataset_scaled)

for i in range(len(test_dataset_scaled)):
    test_emotions.iloc[i] = [test_emotions_p[i], test_emotions_a[i], test_emotions_d[i]]


test_emotions.head()

,EMOTION_P,EMOTION_A,EMOTION_D
37744,0.601657,0.411220,0.084863
33979,0.594111,0.401634,0.084477
69166,0.289399,0.386997,0.450212
37125,0.596537,0.406971,0.137500
74147,0.302023,0.413434,0.664359


In [13]:
test_emotions_OG.head()

,EMOTION_P,EMOTION_A,EMOTION_D
37744,0.6,0.4,0.1
33979,0.6,0.4,0.1
69166,0.3,0.4,0.6
37125,0.6,0.4,0.1
74147,0.3,0.4,0.6


In [14]:
test_dataset.head()

,max_hand_distance,avg_l_hand_hip_distance,avg_r_hand_hip_distance,max_stride_length,avg_l_hand_chest_distance,avg_r_hand_chest_distance,avg_l_elbow_hip_distance,avg_r_elbow_hip_distance,avg_chest_pelvis_distance,avg_neck_chest_distance,...,l_hand_speed,r_hand_speed,l_foot_speed,r_foot_speed,neck_speed,l_hand_acceleration_magnitude,r_hand_acceleration_magnitude,l_foot_acceleration_magnitude,r_foot_acceleration_magnitude,neck_acceleration_magnitude
37744,0.985932,0.517800,0.499155,0.594833,0.521641,0.480978,0.336280,0.307784,0.236151,0.223894,...,-0.084022,-0.084976,-8.180281e-02,-0.052490,-0.087709,0.016990,0.012500,0.020849,0.008774,0.007178
33979,0.287193,0.219668,0.056832,0.348576,0.322342,0.269939,0.145520,0.112937,0.236151,0.223894,...,-0.145636,-0.020754,-3.157135e-02,-0.104433,-0.070317,0.039563,0.023456,0.011433,0.019924,0.008946
69166,0.241711,0.239997,0.017452,0.299543,0.255472,0.233677,0.042455,0.106840,0.236151,0.223894,...,-0.000000,-0.000000,-7.450581e-09,-0.000000,-0.000000,0.056962,0.158072,0.158967,0.055895,0.107948
37125,0.941847,0.527478,0.440791,0.657772,0.492420,0.473166,0.322670,0.297927,0.236151,0.223894,...,-0.089843,-0.119538,-6.369931e-02,-0.059833,-0.102461,0.012249,0.015542,0.014328,0.011440,0.007461
74147,0.715660,0.356332,0.352440,0.461035,0.409411,0.425831,0.325670,0.303043,0.236151,0.223894,...,-0.053529,-0.090967,-1.112816e-01,-0.041377,-0.069376,0.009917,0.006300,0.013451,0.005930,0.007182


In [25]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
#from keras.wrappers.scikit_learn import KerasRegressor
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [28]:
def baseline_model():
    model = keras.Sequential()
    model.add(layers.Dense(3, input_dim=3, kernel_initializer='normal', activation='relu'))
    model.add(layers.Dense(8,kernel_initializer='normal', activation='relu'))
    model.add(layers.Dense(16,kernel_initializer='normal', activation='relu'))
    #model.add(layers.Dense(25,kernel_initializer='normal', activation='relu'))
    model.add(layers.Dense(25,kernel_initializer='normal', activation='linear'))
    
    opt = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt)
    
    return model

In [29]:
estimator = KerasRegressor(model=baseline_model, epochs=100, batch_size=16, 
                           validation_split=0.1, loss="mean_squared_error" 
                           )
kfold = KFold(n_splits=10)
#results = cross_val_score(estimator, train_emotions, train_dataset, cv=kfold)
#print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

In [30]:
estimator.fit(train_emotions, train_dataset)

Epoch 1/100
3535/3535 [==============================] - 3s 900us/step - loss: 0.0079 - val_loss: 0.0069
Epoch 2/100
3535/3535 [==============================] - 3s 889us/step - loss: 0.0069 - val_loss: 0.0068
Epoch 3/100
3535/3535 [==============================] - 3s 879us/step - loss: 0.0069 - val_loss: 0.0069
Epoch 4/100
3535/3535 [==============================] - 3s 867us/step - loss: 0.0069 - val_loss: 0.0068
Epoch 5/100
3535/3535 [==============================] - 3s 866us/step - loss: 0.0068 - val_loss: 0.0068
Epoch 6/100
3535/3535 [==============================] - 3s 867us/step - loss: 0.0069 - val_loss: 0.0068
Epoch 7/100
3535/3535 [==============================] - 3s 879us/step - loss: 0.0068 - val_loss: 0.0068
Epoch 8/100
3535/3535 [==============================] - 3s 893us/step - loss: 0.0068 - val_loss: 0.0068
Epoch 9/100
3535/3535 [==============================] - 3s 894us/step - loss: 0.0068 - val_loss: 0.0068
Epoch 10/100
3535/3535 [==============================]

3535/3535 [==============================] - 3s 880us/step - loss: 0.0066 - val_loss: 0.0066
Epoch 79/100
3535/3535 [==============================] - 3s 883us/step - loss: 0.0066 - val_loss: 0.0067
Epoch 80/100
3535/3535 [==============================] - 3s 865us/step - loss: 0.0066 - val_loss: 0.0066
Epoch 81/100
3535/3535 [==============================] - 3s 868us/step - loss: 0.0066 - val_loss: 0.0066
Epoch 82/100
3535/3535 [==============================] - 3s 888us/step - loss: 0.0066 - val_loss: 0.0066
Epoch 83/100
3535/3535 [==============================] - 3s 893us/step - loss: 0.0066 - val_loss: 0.0066
Epoch 84/100
3535/3535 [==============================] - 3s 904us/step - loss: 0.0066 - val_loss: 0.0066
Epoch 85/100
3535/3535 [==============================] - 3s 900us/step - loss: 0.0066 - val_loss: 0.0066
Epoch 86/100
3535/3535 [==============================] - 3s 880us/step - loss: 0.0066 - val_loss: 0.0066
Epoch 87/100
3535/3535 [==============================] - 3

KerasRegressor(
	model=<function baseline_model at 0x7f4c6815d200>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=mean_squared_error
	metrics=None
	batch_size=16
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.1
	shuffle=True
	run_eagerly=False
	epochs=100
)

## Test

In [31]:
generated = estimator.predict(test_emotions)

print(generated)

982/982 [==============================] - 1s 501us/step
[[0.6579329  0.3659178  0.3316489  ... 0.02246404 0.02178895 0.0129698 ]
 [0.65426433 0.35981792 0.3278091  ... 0.02199955 0.02129056 0.01264809]
 [0.572518   0.32196146 0.30091545 ... 0.02061778 0.01942813 0.01203361]
 ...
 [0.56132007 0.31643814 0.2988903  ... 0.01685086 0.01563882 0.00934209]
 [0.45943564 0.2834291  0.27012616 ... 0.01172519 0.00990353 0.00553712]
 [0.5074698  0.29330003 0.27734193 ... 0.0148336  0.01322764 0.00762521]]


In [32]:
mae_errors = mean_absolute_error(test_dataset, generated, multioutput='raw_values')
mse_errors = mean_squared_error(test_dataset, generated, multioutput='raw_values')

features = ["max_hand_distance",
          "avg_l_hand_hip_distance",
          "avg_r_hand_hip_distance",
          "max_stride_length",
          "avg_l_hand_chest_distance",
          "avg_r_hand_chest_distance",
          "avg_l_elbow_hip_distance",
          "avg_r_elbow_hip_distance",
          "avg_chest_pelvis_distance",
          "avg_neck_chest_distance",
          
          "avg_total_body_volume",
          "avg_lower_body_volume",
          "avg_upper_body_volume",
          
          "avg_triangle_area_hands_neck",
          "avg_triangle_area_feet_hips",
          
          "l_hand_speed",
          "r_hand_speed",
          "l_foot_speed",
          "r_foot_speed",
          "neck_speed",
          
          "l_hand_acceleration_magnitude",
          "r_hand_acceleration_magnitude",
          "l_foot_acceleration_magnitude",
          "r_foot_acceleration_magnitude",
          "neck_acceleration_magnitude",
          
         ]

print("Overall MAE: " + str(mean_absolute_error(test_dataset, generated)))

print()
for i in range(len(mse_errors)):
    print("==" + features[i] + "==")
    print("MSE: %.5f" % mse_errors[i])
    print("MAE: %.5f" % mae_errors[i])
    print("Example [Regen-Real]: " + str(generated[i][i]) + " - " + str(test_dataset.iloc[i,i]))
    print()

Overall MAE: 0.052715267571391194

==max_hand_distance==
MSE: 0.03234
MAE: 0.14574
Example [Regen-Real]: 0.6579329 - 0.9859324005805434

==avg_l_hand_hip_distance==
MSE: 0.01544
MAE: 0.09695
Example [Regen-Real]: 0.35981792 - 0.2196684826856563

==avg_r_hand_hip_distance==
MSE: 0.01445
MAE: 0.09454
Example [Regen-Real]: 0.30091545 - 0.0174522653431972

==max_stride_length==
MSE: 0.02574
MAE: 0.13331
Example [Regen-Real]: 0.5005291 - 0.6577717367618382

==avg_l_hand_chest_distance==
MSE: 0.00838
MAE: 0.07636
Example [Regen-Real]: 0.35828117 - 0.4094106148548246

==avg_r_hand_chest_distance==
MSE: 0.00692
MAE: 0.07057
Example [Regen-Real]: 0.3243177 - 0.4540335559445053

==avg_l_elbow_hip_distance==
MSE: 0.00948
MAE: 0.08558
Example [Regen-Real]: 0.2182308 - 0.1353452043120135

==avg_r_elbow_hip_distance==
MSE: 0.00755
MAE: 0.07564
Example [Regen-Real]: 0.24851376 - 0.3314250392901335

==avg_chest_pelvis_distance==
MSE: 0.00000
MAE: 0.00000
Example [Regen-Real]: 0.23615114 - 0.2361509554

In [33]:
generated_scaled = scaler.transform(generated)

gen_emotions_p = model_p.predict(generated_scaled)
gen_emotions_a = model_a.predict(generated_scaled)
gen_emotions_d = model_d.predict(generated_scaled)

rows = []
for i in range(len(generated_scaled)):
    rows.append([gen_emotions_p[i], gen_emotions_a[i], gen_emotions_d[i]])

gen_emotions = pd.DataFrame(rows, columns=[
            "EMOTION_P", "EMOTION_A", "EMOTION_D"
         ])

gen_emotions.head()

/home/diogosilva/.local/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


,EMOTION_P,EMOTION_A,EMOTION_D
0,0.363631,-0.085778,-0.144262
1,0.359867,-0.163549,-0.179380
2,0.201095,-0.421954,-0.132863
3,0.287717,-0.002271,-0.161169
4,0.233324,-0.321743,-0.091247


In [34]:
test_emotions.head()

,EMOTION_P,EMOTION_A,EMOTION_D
37744,0.601657,0.411220,0.084863
33979,0.594111,0.401634,0.084477
69166,0.289399,0.386997,0.450212
37125,0.596537,0.406971,0.137500
74147,0.302023,0.413434,0.664359


In [35]:
import random

mae_errors = mean_absolute_error(test_emotions, gen_emotions, multioutput='raw_values')
mse_errors = mean_squared_error(test_emotions, gen_emotions, multioutput='raw_values')

features = ["PLEASURE", "AROUSAL", "DOMINANCE"
         ]

print("Overall MAE: " + str(mean_absolute_error(test_emotions, gen_emotions)))

print()
for i in range(len(mse_errors)):
    print("==" + features[i] + "==")
    print("MSE: %.5f" % mse_errors[i])
    print("MAE: %.5f" % mae_errors[i])
    print()
    
for i in range(30):
    row = random.randint(0, len(test_emotions))

    print("Real: " + str([test_emotions_OG.iloc[row,0], test_emotions_OG.iloc[row,1], test_emotions_OG.iloc[row,2]]))
    print("Predicted: " + str([test_emotions.iloc[row,0], test_emotions.iloc[row,1], test_emotions.iloc[row,2]]))
    print("Generated: " + str([gen_emotions.iloc[row,0], gen_emotions.iloc[row,1], gen_emotions.iloc[row,2]]))
    print()

Overall MAE: 0.2929357698059456

==PLEASURE==
MSE: 0.04793
MAE: 0.17349

==AROUSAL==
MSE: 0.26084
MAE: 0.40808

==DOMINANCE==
MSE: 0.14726
MAE: 0.29725

Real: [0.2, -0.75, -0.3]
Predicted: [0.18797975778579712, -0.7553421854972839, -0.2875642478466034]
Generated: [0.07369677, -0.41755444, -0.16253765]

Real: [0.1, 0.6, 0.4]
Predicted: [0.13550616800785065, 0.5777843594551086, 0.4885331392288208]
Generated: [0.29563174, 0.11516759, -0.07323479]

Real: [0.6, 0.4, 0.1]
Predicted: [0.6063784956932068, 0.4221786558628082, 0.09556511044502258]
Generated: [0.3427663, -0.093943916, -0.12038561]

Real: [0.6, 0.4, 0.1]
Predicted: [0.6184388399124146, 0.43794625997543335, 0.10961724817752838]
Generated: [0.29452333, 0.043575153, -0.093963295]

Real: [-0.1, -0.55, -0.15]
Predicted: [-0.10281143337488174, -0.5468388795852661, -0.14911708235740662]
Generated: [0.12829301, -0.5312034, -0.18333836]

Real: [0.3, 0.4, 0.6]
Predicted: [0.3007582724094391, 0.39686471223831177, 0.6267416477203369]
Generate